In [1]:
import csv
import numpy as np
from sklearn.metrics import log_loss
import torch
from torch import autograd,nn, optim
import torch.nn.functional as F

pathToData = "data/"

ModuleNotFoundError: No module named 'torch'

In [447]:
SeasonList = [2005]
    
#with open('data-2005/team-game-statistics.csv', 'r') as f:
 # reader = csv.reader(f)
  #TeamGameStatistics = list(reader)

In [448]:
def ReadDataForASeason(year):
    with open(pathToData + str(year) +'/game.csv', 'r') as f:
        reader = csv.reader(f)
        Game = list(reader)
        Game = Game[1:]

    with open(pathToData + str(year) + '/game-statistics.csv', 'r') as f:
        reader = csv.reader(f)
        GameStatistics = list(reader)
        GameStatistics = GameStatistics[1:]
    
    with open(pathToData + str(year) + '/team-game-statistics.csv', 'r') as f:
        reader = csv.reader(f)
        TeamGameStatistics = list(reader)
        TeamGameStatistics = TeamGameStatistics[1:]
        
    with open(pathToData + str(year) + '/conference.csv', 'r') as f:
        reader = csv.reader(f)
        Conference = list(reader)
        Conference = Conference[1:]
        
    with open(pathToData + str(year) + '/drive.csv', 'r') as f:
        reader = csv.reader(f)
        Drive = list(reader)
        Drive = Drive[1:]
                
    with open(pathToData + str(year) + '/kickoff.csv', 'r') as f:
        reader = csv.reader(f)
        Kickoff = list(reader)
        Kickoff = Kickoff[1:]
                
    with open(pathToData + str(year) + '/kickoff-return.csv', 'r') as f:
        reader = csv.reader(f)
        KickoffReturn = list(reader)
        KickoffReturn = KickoffReturn[1:]
                
    with open(pathToData + str(year) + '/pass.csv', 'r') as f:
        reader = csv.reader(f)
        Pass = list(reader)
        Pass = Pass[1:]
                
    with open(pathToData + str(year) + '/play.csv', 'r') as f:
        reader = csv.reader(f)
        Play = list(reader)
        Play = Play[1:]
                
    with open(pathToData + str(year) + '/player.csv', 'r') as f:
        reader = csv.reader(f)
        Player = list(reader)
        Player = Player[1:]
                
    with open(pathToData + str(year) + '/player-game-statistics.csv', 'r') as f:
        reader = csv.reader(f)
        PlayerGameStatistics = list(reader)
        PlayerGameStatistics = PlayerGameStatistics[1:]
                
    with open(pathToData + str(year) + '/punt.csv', 'r') as f:
        reader = csv.reader(f)
        Punt = list(reader)
        Punt = Punt[1:]
                
    with open(pathToData + str(year) + '/punt-return.csv', 'r') as f:
        reader = csv.reader(f)
        PuntReturn = list(reader)
        PuntReturn = PuntReturn[1:]
                
    with open(pathToData + str(year) + '/reception.csv', 'r') as f:
        reader = csv.reader(f)
        Reception = list(reader)
        Reception = Reception[1:]
                
    with open(pathToData + str(year) + '/rush.csv', 'r') as f:
        reader = csv.reader(f)
        Rush = list(reader)
        Rush = Rush[1:]
                
    with open(pathToData + str(year) + '/stadium.csv', 'r') as f:
        reader = csv.reader(f)
        Stadium = list(reader)
        Stadium = Stadium[1:]
                
    with open(pathToData + str(year) + '/team.csv', 'r') as f:
        reader = csv.reader(f)
        Team = list(reader)
        Team = Team[1:]
        
    return (Game,GameStatistics,TeamGameStatistics)

In [449]:
for year in SeasonList:
    Game, GameStatistics, TeamGameStatistics = ReadDataForASeason(year)

In [450]:
GameStatistics[2]

['0204014020050901', '21458', '197']

In [451]:
#Assuming no anomolies in game data in TEAMGAMESTATS correspnoding to games in GAME
#Add fields from TEAMSGAMESTATS in this function
def SearchGame(GameID, TeamGameStatistics, HomeTeam, VisitTeam):
    count = 0
    for i,gamestats in enumerate(TeamGameStatistics):
        if(gamestats[1] == GameID):
            if(gamestats[0] == HomeTeam):
                HomeTeamStats = gamestats
            else:
                VisitTeamStats = gamestats
            count = count + 1
        if(count == 2):
            return(HomeTeamStats,VisitTeamStats)
    print("Loop ended without returning! :( ")

In [452]:
def listOfDictionaries(GameList, Game, GameStatistics, TeamGameStatistics):
    newGame = {}
    newGame['Date'] = Game[1]
    newGame['GameID'] = Game[0]
    newGame['Attendance'] = GameStatistics[1]
    #print(GameStatistics[2])
    newGame['Duration'] = GameStatistics[2]
    newGame['HomeTeam'] = int(Game[3])
    newGame['VisitTeam'] = int(Game[2])
    HomeTeamStats, VisitTeamStats = SearchGame(Game[0],TeamGameStatistics,Game[3],Game[2])
    newGame['HT-Points'] = int(HomeTeamStats[35])
    newGame['VT-Points'] = int(VisitTeamStats[35])
    GameList.append(newGame)
    return GameList

In [453]:
GameList = [] #List of dictionaries, based on sorted order of games
TeamList = [] #List of Teams in sorted order of Team codes
count = 0
for i in range(len(Game)):
    GameList = listOfDictionaries(GameList, Game[i], GameStatistics[i], TeamGameStatistics)
    #GameList
    if(GameList[i]['HomeTeam'] in TeamList):
        count = 1
    else:
        TeamList.append(GameList[i]['HomeTeam'])
    if(GameList[i]['VisitTeam'] in TeamList):
        count = 1
    else:
        TeamList.append(GameList[i]['VisitTeam'])
        
TeamList.sort()

In [454]:
GameList[6]

{'Attendance': '19981',
 'Date': '09/01/2005',
 'Duration': '235',
 'GameID': '0529028820050901',
 'HT-Points': 24,
 'HomeTeam': 288,
 'VT-Points': 38,
 'VisitTeam': 529}

In [455]:
NumberOfWins = []
NumberOfMatches = []
PointDifference = []
for i,team in enumerate(TeamList):
    NumberOfWins.append(0)
    NumberOfMatches.append(0)
    PointDifference.append(0)

In [456]:
#win ratios of home team, win ratio of visit team
X_train = []
X_test = []
#1 if home team wins, zero otherwise
Y_train = []
Y_test = []

In [457]:
#Skip first 100 games
gamecount = 0
for i,game in enumerate(GameList[:100]):
    gamecount = gamecount + 1
    htindex = TeamList.index(game['HomeTeam'])
    vtindex = TeamList.index(game['VisitTeam'])
    NumberOfMatches[htindex] = NumberOfMatches[htindex] + 1
    NumberOfMatches[vtindex] = NumberOfMatches[vtindex] + 1
    point_difference = game['HT-Points'] - game['VT-Points']
    PointDifference[htindex] += point_difference
    PointDifference[vtindex] -= point_difference 
    if(game['HT-Points'] >= game['VT-Points']):
        NumberOfWins[htindex] = NumberOfWins[htindex] + 1
    else:
        NumberOfWins[vtindex] = NumberOfWins[vtindex] + 1        
    

In [458]:
for i,team in enumerate(TeamList):
    if(NumberOfMatches[i] < NumberOfWins[i]):
        print("Matches: ", NumberOfMatches[i], " Wins: ", NumberOfWins[i])

In [459]:
gamecount = 0
for i,game in enumerate(GameList[100:]):
    temp = []
    gamecount = gamecount + 1
    print("For ",i,": ")
    #print(game)
    htindex = TeamList.index(game['HomeTeam'])
    vtindex = TeamList.index(game['VisitTeam'])
    print(NumberOfWins[htindex])
    print(NumberOfMatches[htindex])
    print(NumberOfWins[vtindex])
    print(NumberOfMatches[vtindex])
    if(NumberOfMatches[htindex] > 0):
        temp.append(NumberOfWins[htindex]/NumberOfMatches[htindex])
    else:
        temp.append(0.0)
    if(NumberOfMatches[vtindex] > 0):
        temp.append(NumberOfWins[vtindex]/NumberOfMatches[vtindex])
    else:
        temp.append(0.0)
    temp.append(PointDifference[htindex])
    temp.append(PointDifference[vtindex])
    X_train.append(temp)
    if(game['HT-Points'] >= game['VT-Points']):
        Y_train.append(1)
        NumberOfWins[htindex] = NumberOfWins[htindex] + 1
    else:
        Y_train.append(0)
        NumberOfWins[vtindex] = NumberOfWins[vtindex] + 1
    point_difference = game['HT-Points'] - game['VT-Points']
    PointDifference[htindex] += point_difference
    PointDifference[vtindex] -= point_difference 
    NumberOfMatches[vtindex] = NumberOfMatches[vtindex] + 1
    NumberOfMatches[htindex] = NumberOfMatches[htindex] + 1

For  0 : 
1
1
1
1
For  1 : 
0
1
0
0
For  2 : 
1
1
0
0
For  3 : 
0
1
0
1
For  4 : 
0
1
0
0
For  5 : 
1
1
0
1
For  6 : 
0
1
0
0
For  7 : 
0
1
1
1
For  8 : 
1
1
1
1
For  9 : 
0
1
0
1
For  10 : 
1
1
0
0
For  11 : 
1
1
0
0
For  12 : 
1
1
1
1
For  13 : 
0
1
0
0
For  14 : 
0
1
1
1
For  15 : 
1
1
0
1
For  16 : 
0
0
0
0
For  17 : 
1
1
0
1
For  18 : 
1
1
0
0
For  19 : 
1
2
2
2
For  20 : 
1
1
1
2
For  21 : 
0
2
0
1
For  22 : 
1
2
2
2
For  23 : 
2
2
1
2
For  24 : 
1
1
1
2
For  25 : 
1
2
0
2
For  26 : 
0
1
2
2
For  27 : 
2
2
1
1
For  28 : 
1
1
0
0
For  29 : 
1
2
0
1
For  30 : 
2
2
1
2
For  31 : 
2
2
2
2
For  32 : 
0
2
1
1
For  33 : 
0
1
0
0
For  34 : 
1
2
1
2
For  35 : 
2
2
0
2
For  36 : 
2
2
2
2
For  37 : 
2
2
0
0
For  38 : 
0
2
0
2
For  39 : 
2
2
2
2
For  40 : 
1
2
0
0
For  41 : 
2
2
1
2
For  42 : 
2
2
0
1
For  43 : 
0
1
0
0
For  44 : 
1
2
0
2
For  45 : 
2
2
0
1
For  46 : 
2
2
2
2
For  47 : 
0
0
1
2
For  48 : 
2
2
1
1
For  49 : 
0
1
1
2
For  50 : 
2
2
0
2
For  51 : 
2
2
0
2
For  52 : 
1
2
0
0
For

In [460]:
X_train[500]

[0.3333333333333333, 0.5555555555555556, -83, 24]

In [461]:
Y_train[500]

1

In [462]:
X_test = X_train[400:]
X_train = X_train[:400]
Y_test = Y_train[400:]
Y_train = Y_train[:400]

In [463]:
batch_size = gamecount
input_size = 4
hidden_size = 200
num_classes = 1
learning_rate = 0.001

#input = autograd.Variable(torch.rand(batch_size,input_size)
input = autograd.Variable(torch.Tensor(X_train))

#input[0][1] = 2
target = autograd.Variable(torch.Tensor(Y_train).long())

In [464]:
i = 0
print(X_train[i])
print(input[i])
print(target[i])
print(Y_train[i])

[1.0, 1.0, 35, 9]
Variable containing:
  1
  1
 35
  9
[torch.FloatTensor of size 4]

Variable containing:
 1
[torch.LongTensor of size 1]

1


In [465]:
len(Y_test)

218

In [466]:
len(Y_train)

400

In [467]:
len(X_train)

400

In [468]:
len(X_test)

218

In [469]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size,num_classes):
        super().__init__()
        #create hidden layers and assign it to self
        self.h1 = nn.Linear(input_size,hidden_size)
        self.h2 = nn.Linear(hidden_size,num_classes)
    def forward(self,x):
        #run forward propogation
        x = self.h1(x)
        x = F.tanh(x)
        x = self.h2(x)
        x = F.sigmoid(x)
        return x

In [470]:
model = Net(input_size = input_size, hidden_size = hidden_size, num_classes = num_classes)
opt = optim.Adam(params=model.parameters(),lr = learning_rate)

In [471]:
out.size()

torch.Size([400, 2])

In [520]:
for epoch in range(10000):
    out = model(input)
    #_, pred = out.max(1)
   # print("Target: ",str(target.view(1,-1)).split('\n')[1])
   # print("Pred: ", str(pred.view(1,-1)).split('\n')[1])
    #loss = F.binary_cross_entropy(out,target.float())
    loss = F.nll_loss(out,target)
    print("Loss: ",loss.data[0])
    model.zero_grad()
    loss.backward()
    opt.step()

RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at c:\anaconda2\conda-bld\pytorch_1519501749874\work\torch\lib\thnn\generic/ClassNLLCriterion.c:87

In [490]:
print("Target: ",str(target.view(1,-1)).split('\n')[1])
print("Pred: ", str(pred.view(1,-1)).split('\n')[1])

Target:  
Pred:  


In [502]:
pred = np.round(out.data.numpy())

In [503]:
target[350]

Variable containing:
 0
[torch.LongTensor of size 1]

In [504]:
torch.save(model.state_dict(), "mymodel")

In [505]:
def Accuracy(Y_train,Yhat):
    CorrectPredictions = 0
    print(len(Y_train))
    print(len(Yhat))
    for i,current in enumerate(Y_train):
        if((Y_train[i] == 0 and Yhat[i] < 0.5) or (Y_train[i] == 1 and Yhat[i] > 0.5)):
            CorrectPredictions = CorrectPredictions + 1
    return(100 * CorrectPredictions/len(Y_train), CorrectPredictions)


In [506]:
t = target.data.numpy()

In [510]:
accuracy, CorrectPredictions = Accuracy(t,pred)

400
400


In [511]:
CorrectPredictions

398

In [512]:
accuracy

99.5

In [513]:
the_model = Net(input_size = input_size, hidden_size = hidden_size, num_classes = num_classes)
the_model.load_state_dict(torch.load("mymodel"))

In [514]:
test_input = autograd.Variable(torch.Tensor(X_test))
test_target = autograd.Variable(torch.Tensor(Y_test).long())

In [515]:
out2 = the_model(test_input)
_, pred2 = out2.max(1)

In [516]:
pred2

Variable containing:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
[torch.LongTensor of size 218]

In [517]:
accuracytest, CorrectPredictionstest = Accuracy(test_target.data.numpy(), pred2.data.numpy())

218
218


In [519]:
accuracytest

44.95412844036697

In [392]:
CorrectPredictionstest

145